In [8]:
#Import Library
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator #type: ignore
from tensorflow.keras.applications import MobileNetV2 #type: ignore
from tensorflow.keras.models import Sequential #type: ignore
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout #type: ignore
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input #type: ignore
import json
import os

In [9]:
# Konfig
DATASET_PATH = 'dataset/'
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10 

#Augmentasi
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    os.path.join(DATASET_PATH, 'train/'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    os.path.join(DATASET_PATH, 'val/'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

class_names = list(train_generator.class_indices.keys())
num_classes = len(class_names)

Found 185 images belonging to 2 classes.
Found 44 images belonging to 2 classes.


In [10]:
# Arsitektur Model 
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False 
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,914,882 (11.12 MB)

 Trainable params: 656,898 (2.51 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [11]:
#train model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
).history

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 586ms/step - accuracy: 0.6145 - loss: 0.8845 - val_accuracy: 1.0000 - val_loss: 0.0250
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9375 - loss: 0.1332 - val_accuracy: 1.0000 - val_loss: 0.0223
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 349ms/step - accuracy: 0.9839 - loss: 0.0877 - val_accuracy: 1.0000 - val_loss: 0.0034
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.9375 - loss: 0.1669 - val_accuracy: 1.0000 - val_loss: 0.0063
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 377ms/step - accuracy: 0.9428 - loss: 0.0947 - val_accuracy: 1.0000 - val_loss: 0.0019
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.9688 - loss: 0.0525 - val_accuracy: 1.0000 - val_loss: 0.0017
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 368ms/step - accuracy: 0.9693 - loss: 0.0824 - val_accuracy: 1.0000 - val_loss: 0.0026
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.9688 - loss: 0.0509 - val_accuracy: 1.0000 - val_loss: 0.0

In [12]:
# save model
model.save('model1.h5')
with open('class_names.json', 'w') as f:
    json.dump(class_names, f)

print("Done disimpan")

Done disimpan


In [13]:
#Uji coba klasifikasi
from tensorflow.keras.models import load_model # type: ignore
from tensorflow.keras.preprocessing import image # type: ignore
import numpy as np
import json
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input # type: ignore

#load model
MODEL_PATH = 'model1.h5'
loaded_model = load_model(MODEL_PATH)

with open('class_names.json', 'r') as f:
    class_names = json.load(f)

def predict_image(image_path):

    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    processed_img_array = preprocess_input(img_array) 

    predictions = loaded_model.predict(processed_img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_name = class_names[predicted_class_index]
    confidence = np.max(predictions[0]) * 100

    return predicted_class_name, confidence

#ujinya
TEST_IMAGE_PATH = 'dataset/ujicoba/jb2.jpg' 

try:
    predicted_class, confidence_score = predict_image(TEST_IMAGE_PATH)
    print(f"Gambar: {TEST_IMAGE_PATH}")
    print(f"Hasil Prediksi: {predicted_class}")
    print(f"Confidence: {confidence_score:.2f}%")
except FileNotFoundError:
    print(f"File not found di {TEST_IMAGE_PATH}")
except NameError as e:
    print(f"error{e}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 555ms/step
Gambar: dataset/ujicoba/jb2.jpg
Hasil Prediksi: jembatan
Confidence: 99.86%
